In [1]:
!pip install -r requirements.txt 

## Exemplo de classificador com leitura do CSV. 

Etapas:

- Leitura CSV.
- Tratamento de imagens com lib PIL. 
- Leitura imagem. 
- Aplicacao Filtro para remocao background


Input do modelo : (Imagem, sexo, idade, etnia)

In [2]:
## Criacao de pastas output e models se nao existirem
import os
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('models'):
    os.makedirs('models')

In [3]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim
import os

In [4]:
## Geracao dataset com aleatorio
ds_file = pd.read_csv('dataset/exemplo_csv_2.csv')
arr = np.random.choice([0, 1], size=216)

ds_file = ds_file[['path','image_id','ID','IDADE','SEXO','ETNIA']]
ds_file['oca'] = arr
ds_file.to_csv('dataset/oca.csv')

In [5]:
ds_file = pd.read_csv('dataset/oca.csv')


## Geracao Tensor com imagens filtradas

In [6]:
## Loop
img_dataset = np.ones((ds_file.shape[0],3,256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,2)
    j+=1
tensor_imagem = torch.tensor(img_dataset)



## Geracao Tensor Age, sex, Etinia

Encoding: 

- Sexo: M=0 F=1

- Idade: Normalizar? Sim, dividir por 100 igual o artigo do Marco e Felipe fizeram. 

- Etnia: Encoding simples de categorias. Utilizaremos o Label Encoding para isso 

In [7]:
ds_id_sex_et= ds_file[['IDADE', 'SEXO', 'ETNIA']]
ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
ds_id_sex_et['IDADE'] = ds_file['IDADE']/100

In [8]:
## Encoding variavel ETNIA -> Usando label Encoding
from sklearn.preprocessing import LabelEncoder

# Create a sample dataframe with categorical data

#print(f"Before Encoding the Data:\n\n{ds_id_sex_et['ETNIA']}\n")

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the categorical data
ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])
ds_id_sex_et

,IDADE,SEXO,ETNIA
0,0.85,0,1
1,0.32,1,1
2,0.60,1,1
3,0.57,1,4
4,0.57,1,1
...,...,...,...
211,0.35,1,3
212,0.72,0,2
213,0.71,1,2
214,0.49,0,2


## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ OCA, NOCA]

In [9]:
##
ds_file.columns
labels = [ 'oca']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))
tensor_label

tensor([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label

## Criacao Subset 



In [10]:
class Subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    @property
    def classes(self):
        return self.data.classes

    def shape(self):
        return self.dataset

## K-fold = 10


In [11]:
kf = KFold(n_splits=5)
kf.get_n_splits(tensor_imagem)
print(kf)
#for i in enumerate(kf.split(tensor_imagem)):
#    print(i)

KFold(n_splits=5, random_state=None, shuffle=False)


## Criacao do dataset de treino e validacao com batchsize

In [12]:
BATCH_SIZE = 16
train_dataset = TensorDataset(tensor_imagem, tensor_label)
train_loader_img = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [24]:
tensor_label

tensor([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [0],

In [13]:
train_dataset[0][0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

## Create a model to concat layers

O modelo utilizado será o mesmo do artigo: Artificial Intelligence-Driven Screening System for Rapid Image-Based Classification of 12-Lead ECG Exams: A Promising Solution for Emergency Room Prioritization 

Desenvolvido conforme a figura abaixo: 

![image.png](attachment:image.png)

In [14]:
#from models import ECGClassifierResnet

class ECGClassifierResnet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True) 
        self.base_model=timm.create_model('resnet50d.ra4_e3600_r224_in1k',pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=5,pretrained=True)


        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 2048        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size,1)
        ) # saida como Sigmoid multilabel

    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        #output = nn.Softmax(dim=1)(x)
        return output

## Train - Valid Loop

Fluxo de treinamento

Input: modelo, dataloader_train, dataloader_teste , epocas.
- Para cada epoca

    - treinamento:

        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)
            
            leitura das imagens e rotulos. 
            
            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss () 
            
            aplica o backward -> Ajuste dos pesos nos neuronios de acordo com o valor de loss (w_{i})

            otimizo o modelo com optimizer.step()

    - validacao (avaliacao do desempenho para aquele conjunto de treinamento):
        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)

            leitura das imagens e rotulos.

            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss ()


    - Calculo de metricas de loss media de validacao.

    - Calculo de metrica de loss media de treinamento. 

    - Print de metricas de desempenho para a epoca. 








In [15]:
np.array([np.empty(5)], dtype=int)

array([[0, 0, 0, 0, 0]])

In [16]:
np.empty(5)

array([0.0e+000, 4.9e-324, 9.9e-324, 1.5e-323, 2.0e-323])

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))
def simple_loop(model, train_image, val_image, epochs):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5
    iter_size = 5
    print(f'Number of training images per iteration: {iter_size}')
    #model = modelo( num_classes=5)
    model.to(device)
    criterion =  nn.L1Loss()
    #criterion =  nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    predict_label_full_epoch = []
    predict_label_full = []
    true_label_full_epoch = []
    true_label_full = []
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            #print(outputs)
            #print(label)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        predict_label =[]
        true_label =[]
        _iter=0
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                #print( [outputs.cpu().data.numpy().astype(int).T[0]])
                #print(label.cpu().data.numpy().astype(int).T[0])
                #print(predict_label)
                try:
                    _pred = outputs.cpu().data.numpy().astype(int).T[0].tolist()
                    if(len(_pred) == iter_size):
                        predict_label.append(_pred)
                        _true = label.cpu().data.numpy().astype(int).T[0].tolist()
                        true_label.append(_true)
                except Exception as e:
                    print(f"Concatenation error iter: {e}")
                    print(_pred)
                    print(predict_label)    
                running_loss_valid += loss_valid.item() * label.size(0)
                _iter +=1
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        print(f'End validation for epoch {epoch}')
        print(f'Amount of images validated: {val_image}')
        print(f'Label predict shape : {len(predict_label)} for epoch {epoch}')
        print(f'Count of iterations: {_iter} for epoch {epoch}')
        try:
            _p = predict_label
            _t = true_label
            predict_label_full.append(_p)
            true_label_full.append(_t)
        except Exception as e:
            print(f"Concatenation error full: {e}")
            print(predict_label)
            print(true_label)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {epoch}:')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

    predict_label_full_out = np.array(predict_label_full)
    true_label_full_out = np.array(true_label_full)

    return train_losses, val_losses, model,predict_label_full_out, true_label_full_out

Device: cuda:0


In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))
def simple_loop(model, train_image, val_image, epochs):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5
    iter_size = (train_image.dataset.__len__())
    print(f'Number of training iterations per epoch: {iter_size}')
    #model = modelo( num_classes=5)
    model.to(device)
    criterion =  nn.L1Loss()
    #criterion =  nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    predict_label_full = np.array([[1,1,1,1,1]], dtype=int)
    predict_label_full = np.array([np.ones([5], dtype=int)])
    true_label_full = np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
    true_label_full = np.array([np.ones([5], dtype=int)])
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            #print(outputs)
            #print(label)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        predict_label =np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
        predict_label =np.array([np.empty(5, dtype=int)])
        predict_label =[]
        true_label =np.array([[1,1,1,1,1],[1,1,1,1,1]], dtype=int)
        true_label =np.array([np.empty(5, dtype=int)])
        _iter=0
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                #print( [outputs.cpu().data.numpy().astype(int).T[0]])
                #print(label.cpu().data.numpy().astype(int).T[0])
                #print(predict_label)
                try:
                    _pred = [outputs.cpu().data.numpy().astype(int).T[0]]
                    predict_label.append(_pred)
                  #  print(_pred)
                    if (_iter == 0):
                        predict_label=_pred
                    else:
                        predict_label=np.concatenate((predict_label,_pred), axis=0)
                  #  print(predict_label)
                    _true = [label.cpu().data.numpy().astype(int).T[0]]
                    true_label=np.concatenate((true_label, _true), axis=0)
                except Exception as e:
                    print(f"Concatenation error: {e}")
                    print(_pred)
                    print(predict_label)    
                running_loss_valid += loss_valid.item() * label.size(0)
                _iter +=1
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        print(f'End validation for epoch {epoch}')
        print(f'Amount of images validated: {val_image}')
        print(f'Label predict shape : {len(predict_label)} for epoch {epoch}')
        print(f'Count of iterations: {_iter} for epoch {epoch}')
        try:
            predict_label_full=np.concatenate((predict_label_full,predict_label), axis=0)
            true_label_full=np.concatenate((true_label_full,true_label), axis=0)
        except Exception as e:
            print(f"Concatenation error: {e}")
            print(predict_label)
            print(true_label)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {epoch}:')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")
    
    return train_losses, val_losses, model,predict_label_full, true_label_full

Device: cuda:0


In [19]:
def salvar_model(model, path = 'output/modelos', name_file='model.pth'):
    full_path = os.path.join(path, name_file)
    torch.save(model.state_dict(), full_path)

In [20]:
def salvar_metricas(path='output/metricas', name_file_train='train_loss_total.npy', name_file_val='val_loss_total.npy',predict_label=None, true_label=None):
    full_path_train = os.path.join(path, name_file_train)
    full_path_val = os.path.join(path, name_file_val)
    np.save(full_path_train, np.array(predict_label))
    np.save(full_path_val, np.array(true_label))

In [21]:

train_loss_total = []
val_loss_total =[]
all_models =[]
epochs = 10
## create first model.
for i, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    ## init train test for folder
    train_dataset_part = Subset( train_dataset, train_index)
    val_dataset_part = Subset( train_dataset, test_index)

    train_loader_img = DataLoader(train_dataset_part, batch_size=5, shuffle=True)
    val_loader_img = DataLoader(val_dataset_part, batch_size=5, shuffle=True)

    model= ECGClassifierResnet( num_classes=1)
    salvar_model(model, path='output/modelos', name_file=f'model_fold_{i}.pth')
    print(f'Train and valid for Fold {i}')
    t, l,_,outputs,labels = simple_loop(model, train_loader_img,val_loader_img, epochs)
    ## Evaluate model.
    salvar_metricas(path='output/metricas', name_file_train=f'predict_label_{i}.npy', name_file_val=f'true_label_{i}.npy', predict_label= outputs, true_label= labels)
    train_loss_total.append(t)
    val_loss_total.append(l)

Fold 0:
Train and valid for Fold 0
Number of training iterations per epoch: 172


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
[array([2174, 2174, 2174, 2484, 2174])]
[array([2174, 2174, 4134, 7334, 2174]), [array([2174, 2174, 2174, 2484, 2174])]]
Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
[array([2174, 2241, 2174, 2174, 3527])]
[array([2174, 2174, 4134, 7334, 2174]), [array([2174, 2174, 2174, 2484, 2174])], [array([2174, 2241, 2174, 2174, 3527])]]
Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.
[array([2998, 3879, 2174, 2174, 2174])]
[array([2174, 2174, 4134, 7334, 2174]), [array([2174, 2174, 2174, 2484, 2174])], [array([2174, 2241, 2174, 2174, 3527])], [array([2998, 3

Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

...

Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.
[array([0, 0, 1, 0, 0])]
[array([0, 0, 0, 0, 0]), [array([0, 0, 1, 0, 0])]]
Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.
[array([0, 0, 0, 0, 0])]
[array([0, 0, 0, 0, 0]), [array([0, 0, 1, 0, 0])], [array([0, 0, 0, 0, 0])]]
Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4,) + inhomogeneous part.
[array([0, 0, 0, 0, 0])]
[array([0, 0, 0, 0, 0]), [array([0, 0, 1, 0, 0])], [array([0, 0, 0, 0, 0])], [array([0, 0, 0, 0, 0])]]
Concatenation error: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (

Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
outputs
## formato ddos outputs [epoch][batch][labels]
#out = np.array(outputs)

array([[[31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616],
        [31616, 31616, 31616, 31616, 31616]],

       [[   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17],
        [   17,    17,    17,    17,    17]],

       [[    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
        [    2,     2,     2,     2,     2],
      

In [ ]:
np.array(labels[0][0:-2])

array([[1, 1, 0, 0, 0],
       [0, 1, 1, 1, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 1, 1, 1],
       [0, 0, 1, 1, 1],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 0, 1]])

## Métricas de Avaliacao dos modelos

Acuracia

Precisao

Sensibilidade

Especificidade

F1-Score

In [ ]:
#### Leitura de métricas salvas
import numpy as np
predict_label_0 = np.load('output/metricas/predict_label_0.npy')
true_label_0 = np.load('output/metricas/true_label_0.npy')
# --- IGNORE ---


array([[[5090, 4552, 5090, 5090, 5090],
        [9356, 5090, 3771, 5021, 5090],
        [5090, 5090, 3520, 5090, 5453],
        [5090, 5090, 5090, 9419, 5090],
        [5090, 5090, 5090, 5090, 4520],
        [5090, 5090, 4277, 5090, 5090],
        [3335, 5090, 7920, 5090, 5090],
        [5090, 5090, 5017, 5281, 5090]],

       [[   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[  31,   31,    1,    3,   31],
        [  31,   31,   31,    1,   31],
        [  31,   31,    4,   31,   31],
        [  31,   31,    2,   31,   31],
        [  20,   31,   31,    0,   31],
        [  77,   11,   31,   31,   31],
        [  31,    4,   31,    1,   31],
        [   2,   31,   39,    9,   31]],

       [[   0,    1,    1,    1,  

In [ ]:
220/16

13.75

In [ ]:
print("a")
print(type(predict_label_0))
predict_label_0.shape

a
<class 'numpy.ndarray'>


(102, 5)

## Criacao graficos de treinamento e validacao

In [ ]:
## Criar graficos de treinamento e validacao
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure(figsize=(10,5))
    plt.plot(train_loss_total[i], label='Train Loss')
    plt.plot(val_loss_total[i], label='Validation Loss')
    plt.title(f'Fold {i} - Train and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'output/graficos/loss_fold_{i}.png')
    plt.show()
    plt.close()
